In [1]:
import math
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import sklearn
import torch
from sklearn.decomposition import TruncatedSVD, PCA
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from scipy.special import softmax
from tqdm import tqdm 
from collections import Counter
import warnings
from sklearn.exceptions import ConvergenceWarning
import matplotlib.pyplot as plt
warnings.simplefilter("always", ConvergenceWarning)
import glob
import pickle
import pprint

In [2]:
from maml.datasets.miniimagenet import MiniimagenetMetaDataset, Task
from maml.models.gated_conv_net_original import ImpRegConvModel
from maml.models.conv_embedding_model import RegConvEmbeddingModel
from maml.logistic_regression_utils import logistic_regression_grad_with_respect_to_w, logistic_regression_hessian_pieces_with_respect_to_w, logistic_regression_hessian_with_respect_to_w, logistic_regression_mixed_derivatives_with_respect_to_w_then_to_X
from maml.logistic_regression_utils import logistic_regression_mixed_derivatives_with_respect_to_w_then_to_X_left_multiply
from maml.algorithm import MetaOptnet, ProtoNet, ImpRMAML_inner_algorithm

In [3]:
all_features_files = glob.glob('inner_solvers_features/**')

In [14]:
all_features_files

['inner_solvers_features/minim_5w5s_protonet_features_dict.pkl',
 'inner_solvers_features/minim_5w1s_protonet_features_dict.pkl',
 'inner_solvers_features/minim_5w1s_SVM_features_dict.pkl',
 'inner_solvers_features/minim_5w1s_LR_features_dict.pkl',
 'inner_solvers_features/minim_5w1s_protosvm_features_dict.pkl',
 'inner_solvers_features/minim_5w5s_protosvm_features_dict.pkl',
 'inner_solvers_features/minim_5w5s_LR_features_dict.pkl',
 'inner_solvers_features/minim_5w5s_SVM_features_dict.pkl']

In [4]:
all_features = []
for ff in all_features_files:
    with open(ff, 'rb') as f:
        all_features.append(pickle.load(f))

## modular re-usable methods

In [5]:
def compute_mean_variance(X):
    """
        X: (N x d)
        returns scalar
        sum (||X - mean_X||_2^2) / N
    """
#     print(f"recvd X of shape {X.shape}")
    N = X.shape[0]
    mean = X.mean(0)
    return mean, (np.sum((X - X.mean(0))**2))/N

In [6]:
def get_X_y_from_features_dict(features_dict):
    X = []
    y = []
    for label in features_dict.keys():
        X.append(features_dict[label])
        y += [label] * X[-1].shape[0]
    X = np.concatenate(X, axis=0)
    y = np.array(y)
#     print(f"Finally returning X, y of shapes : {X.shape} and {y.shape}")
    return X, y

In [7]:
def get_PCA(X):
    pca = PCA()
    pca.fit(X)
    return pca

## metrics to evaluate 

In [8]:
# interclass_vs_intraclass variance
def feature_clustering(features_dict, split):
    X, y = get_X_y_from_features_dict(features_dict[split])
    all_labels = set(y)
    means = []
    numerator = 0.
    for label in all_labels:
        mean, numerator_var = compute_mean_variance(X[y==label, :])
        means.append(mean)
        numerator += numerator_var
    _, denominator = compute_mean_variance(np.stack(means, axis=0))
    print("num", numerator)
    print("denom", denominator)
    return (numerator / denominator) / 100

In [9]:
# variance explained by top k components
def variance_explained(features_dict, split):
    X, y = get_X_y_from_features_dict(features_dict[split])
    all_labels = set(y)
    var_explained = []
    for label in all_labels:
        pca = get_PCA(X[y==label, :])
        var_explained.append(pca.explained_variance_ratio_)
    var_explained = (np.stack(var_explained, axis=0)).mean(0)
    return var_explained

In [10]:
# variance in disc direction for classes 65 and 70 (can be averaged over multiple ones)
def variance_discr_direction(features_dict, split):
    X, y = get_X_y_from_features_dict(features_dict[split])
    all_labels = set(y)
    n_runs = 20
    avg_var = 0.
    for _ in range(n_runs):
        binary_problem_labels = np.random.choice(
            list(all_labels), 2, replace=False)
        var_explained = []
        X_1 = X[y==binary_problem_labels[0], :]
        X_2 = X[y==binary_problem_labels[1], :]
        y_bin = np.array([0] * len(X_1) + [1] * len(X_2))
        with warnings.catch_warnings(record=True) as wn:
            lr_classifier = LogisticRegression()
            lr_classifier.fit(np.concatenate([X_1, X_2], axis=0), y_bin)
        _, var_1 = compute_mean_variance(X_1 @ lr_classifier.coef_.T)
        _, var_2 = compute_mean_variance(X_2 @ lr_classifier.coef_.T)
        _, tvar_1 = compute_mean_variance(X_1)
        _, tvar_2 = compute_mean_variance(X_2)
        avg_var += (var_1 / tvar_1 + var_2 / tvar_2) / 2.
    return avg_var / n_runs

In [ ]:
# variaance for each indi. task
def task_variance(features_dict, split):
    X, y = get_X_y_from_features_dict(features_dict[split])
    all_labels = set(y)
    n_runs = 20
    avg_var = 0.
    for _ in range(n_runs):
        binary_problem_labels = np.random.choice(
            list(all_labels), 2, replace=False)
        var_explained = []
        X_1 = X[y==binary_problem_labels[0], :]
        X_2 = X[y==binary_problem_labels[1], :]
        y_bin = np.array([0] * len(X_1) + [1] * len(X_2))
        with warnings.catch_warnings(record=True) as wn:
            lr_classifier = LogisticRegression()
            lr_classifier.fit(np.concatenate([X_1, X_2], axis=0), y_bin)
        _, var_1 = compute_mean_variance(X_1 @ lr_classifier.coef_.T)
        _, var_2 = compute_mean_variance(X_2 @ lr_classifier.coef_.T)
        _, tvar_1 = compute_mean_variance(X_1)
        _, tvar_2 = compute_mean_variance(X_2)
        avg_var += (var_1 / tvar_1 + var_2 / tvar_2) / 2.
    return avg_var / n_runs

## main analysis engine


In [11]:
engine = {
    'fc': feature_clustering,
    'var_exp': variance_explained,
    'var_disc': variance_discr_direction,
    'per_task_variance': task_variance
}


In [12]:
for analysis_name, analysis_func in engine.items():
    metrics = {}
    print(f"Running analysis: {analysis_name}")
    for i, (feature_name, features) in enumerate(zip(all_features_files, all_features)):  
        name = " ".join(feature_name.split('/')[-1].split('.')[0].split('_')[1:3])
        print(name)
        metrics[name] = engine[analysis_name](all_features[i], 'val')
    pprint.pprint(f"Results:") 
    pprint.pprint(metrics)

Running analysis: fc
5w5s protonet
num 45239.603125
denom 408.4081726074219
5w1s protonet
num 17598.850729166665
denom 218.79690551757812
5w1s SVM
num 162.87037923177084
denom 2.6231672763824463
5w1s LR
num 879.0422591145832
denom 12.17524528503418
5w1s protosvm
num 11097.01125
denom 156.21510314941406
5w5s protosvm
num 18832.833749999998
denom 192.88168334960938
5w5s LR
num 1547.010279947917
denom 12.673722267150879
5w5s SVM
num 117.93078328450521
denom 1.5197663307189941
'Results:'
{'5w1s LR': 0.7219914166288729,
 '5w1s SVM': 0.6208920822479224,
 '5w1s protonet': 0.8043464183159014,
 '5w1s protosvm': 0.7103673733381665,
 '5w5s LR': 1.2206439807803153,
 '5w5s SVM': 0.7759797075430189,
 '5w5s protonet': 1.1077056278324309,
 '5w5s protosvm': 0.9763930624695131}
Running analysis: var_exp
5w5s protonet
5w1s protonet
5w1s SVM
5w1s LR
5w1s protosvm
5w5s protosvm
5w5s LR
5w5s SVM
'Results:'
{'5w1s LR': array([1.16410024e-01, 5.69433831e-02, 4.42904644e-02, 3.64640467e-02,
       2.67674066e-

5w1s protonet
5w1s SVM
5w1s LR
5w1s protosvm
5w5s protosvm
5w5s LR
5w5s SVM
'Results:'
{'5w1s LR': 0.2267856137666279,
 '5w1s SVM': 0.6064494570957661,
 '5w1s protonet': 0.026039271080074356,
 '5w1s protosvm': 0.03409649217180535,
 '5w5s LR': 0.1155442691459611,
 '5w5s SVM': 0.8277073128865762,
 '5w5s protonet': 0.012621979419477607,
 '5w5s protosvm': 0.02492306391954976}


In [13]:
for name, value in metrics.items():
    if '5w1s' in name:
        plt.plot(value[:10], label=name, marker='o')
    print(name, sum(value[:300]))
plt.xticks(np.arange(0, 10), size=8)
plt.yticks(np.arange(0, 0.2, 0.02), size=8)
plt.legend()
plt.show()

IndexError: invalid index to scalar variable.